In [1]:
import numpy as np
import torch
import Doc_import
import os
import json
import Data_Preprocessing

In [2]:
from transformers import BertModel

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import Data_Preprocessing

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        kernel = 4
        input_ff = int(393216/(kernel * kernel))
        zwischenlayer1 = int(1000)
        zwischenlayer2 = int(10)
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.pooling = nn.MaxPool2d(kernel)
        self.linear11 = nn.Linear(input_ff, zwischenlayer1)
        self.linear12 = nn.Linear(zwischenlayer1, zwischenlayer2)
        self.linear13 = nn.Linear(zwischenlayer2, 1)

    def forward(self, input_data):
        bert_output = self.bert(**input_data)
        bert_output = self.pooling(bert_output[0])
        bert_output = bert_output[0].view(-1, 24576)
        x2 = F.relu(self.linear11(bert_output))
        x2 = F.relu(self.linear12(x2))
        x2 = self.linear13(x2)

        return x2

    def train_x2(self, features, labels, loss_function, optimizer, epochs):
        log_interval = 5
        for epoch in range(epochs):
            for i, data in enumerate(train_features):
                data = Data_Preprocessing.preprocessing(data)
                optimizer.zero_grad()
                output1 = self.forward(data)

                loss_x2 = loss_function(output1, torch.tensor(labels[i]['multi-author']))
                print(' ist=', output1, ' soll=', torch.tensor(labels[i]['multi-author']), ' Loss=', loss_x2)
                loss_x2.backward(retain_graph=True)
                optimizer.step()
                '''if index % log_interval == 0:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epochs, index * len(features),
                                                                                    len(features),
                                                                                    100. * index / len(features),
                                                                                    loss_x1.data.item()))'''

    def test_model(self, features, labels, loss_function):
        log_interval = 5
        for index, data in enumerate(features):
            output = self(data)
            loss = loss_function(output, labels[index]['multi-author'])

            if index % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(index, index * len(data),len(data),
                                                                               100. * index / len(data),
                                                                               loss.data.item()))
                        

In [3]:
print(Doc_import.import_verification(base_path=r"/Users/alexbeetz/Desktop/pan20-authorship-verification-training-small"))

FileNotFoundError: [Errno 2] No such file or directory: '.DS_Store'

In [10]:
train_features, train_labels = Doc_import.import_verification(base_path="/Users/alexbeetz/Desktop/pan20-authorship-verification-training-small")


NotADirectoryError: [Errno 20] Not a directory: '/Users/alexbeetz/Desktop/pan20-authorship-verification-training-small/2pan20-authorship-verification-training-small-truth.jsonl'

In [ ]:
model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
criterion = nn.L1Loss()
model.train()
model.train_x2(train_features,train_labels , criterion, optimizer, 1)